In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%%time
# load analysed words
analysed_words = pd.read_csv('/home/jvdzwaan/data/tmp/adh/merged/analysed_words.csv', encoding='utf-8')

In [ ]:
analysed_words.head()

In [ ]:
%%time
# select a single book
book = analysed_words[analysed_words['title'] == 'Fiqh2'].copy()

In [ ]:
books = analysed_words.groupby('title')

In [ ]:
book.shape

In [ ]:
book.head()

In [ ]:
senses_roots = list('''سمع
بصر
لمس
شمم
ذوق'''.split('\n'))
# select certain root
root = senses_roots[0]

In [ ]:
# number of words in book
total_words = book.groupby(['file_name', 'position']).count().shape[0]

In [ ]:
%%time


res = []
titles = []

for title, book in books:
    #print(title)
    b = {}
    total_words = book.groupby(['file_name', 'position']).count().shape[0]
    for i, r in enumerate(senses_roots):
        num = book[book['root'] == r].shape[0]
        print(u'sense {} ({}): {} of {}'.format(i, r, num, total_words))
        b['sense {}'.format(i)] = num
    print(sum(b.values()))
    b['total_words'] = total_words-sum(b.values())
    res.append(b)
    titles.append(title)
data = pd.DataFrame(res, index=titles)
data.head()

In [ ]:
data.head()

In [ ]:
data.columns = ['hear', 'see', 'touch', 'smell', 'taste', 'total_words']

In [ ]:
data.plot.barh(figsize=(15,10), stacked=True)

In [ ]:
data[['hear', 'see', 'touch', 'smell', 'taste']].plot.barh(figsize=(15,10), stacked=True)

In [ ]:
query = {'hear': [], 'see': [], 'touch': [], 'smell': [], 'taste': []}
query['hear'].append(senses_roots[0])
query['see'].append(senses_roots[1])
query['touch'].append(senses_roots[2])
query['smell'].append(senses_roots[3])
query['taste'].append(senses_roots[4])

In [ ]:
query

In [ ]:
query = {'hear': ['سمع', 'صوت', 'ضجج'],
 'see': ['بصر', 'نظر', 'رأي'],
 'touch': ['لمس', 'مسس', 'لصق'],
 'smell': ['شمم', 'ريح', 'عطر'],
 'taste': ['ذوق', 'طعم', 'لذذ']}
query

In [ ]:
senses = {'universe': ['كون']}

In [ ]:
def match_roots(row, to_count):
    #print(row)
    roots = row['proposed_root'].split('\\')
    overlap = set(roots).intersection(to_count)
    if len(overlap) > 0:
        return True
    return False

In [ ]:
senses = query
senses

In [ ]:
in_files = get_files(in_dir)
print(in_files)

In [ ]:
import os

from tqdm import tqdm_notebook as tqdm

from nlppln.utils import get_files

from adhtools.utils import analyzer_xml2df

in_dir = '/home/jvdzwaan/Downloads/2019-02-08-fiqh-newfiles-alkhalil/'

in_files = get_files(in_dir)

result = {}

for in_file in tqdm(in_files):
    df = analyzer_xml2df(in_file)
    #print(df.columns)
    res = {}
    for name, s in senses.items():
        to_count = set(s)
        #print(name, s)
        res[name] = df.apply(lambda row: match_roots(row, to_count), axis=1).sum()
    res['total'] = df.shape[0]
    #print(res)
    file_id = os.path.splitext(os.path.basename(in_file))[0]
    #print(file_id)
    result[file_id] = res
    #break
alk = pd.DataFrame.from_dict(result, orient='index')
alk

In [ ]:
alk_dfs = []
for in_file in tqdm(in_files):
    df = analyzer_xml2df(in_file)
    alk_dfs.append(df)

In [ ]:
print(len(alk_dfs))

In [ ]:
import pickle

def pickle_obj(fname, obj):
    with open(fname, 'wb') as f:
        pickle.dump(obj, f)

pickle_obj('alk_dfs.pkl', alk_dfs)

In [ ]:
def save_dfs(dfs, fname):
    for i, df in enumerate(dfs):
        df.to_csv('{}-{}.csv'.format(fname, i))
save_dfs(alk_dfs, 'alkhalil-fiqh')

In [ ]:
result = {}
for df in tqdm(alk_dfs):
    res = {}
    for name, s in senses.items():
        #print(name, s)
        to_count = set(s)
        res[name] = df.apply(lambda row: match_roots(row, to_count), axis=1).sum()
    res['total'] = df.shape[0]
    #print(res)
    file_id = os.path.splitext(os.path.basename(in_file))[0]
    #print(file_id)
    result[file_id] = res
    #break
alk = pd.DataFrame.from_dict(result, orient='index')
alk

In [ ]:
alk.shape

In [ ]:
alk.to_csv('senses-fiqh-0179-alkhalil.csv')

In [ ]:
alk = pd.read_csv('senses-fiqh-new_files-alkhalil.csv')
#alk = pd.read_csv('senses-single-fiqh-new_files-alkhalil.csv')
alk = alk.rename(columns={'Unnamed: 0': 'BookURI'})
alk = alk.set_index('BookURI')
alk

In [ ]:
senses

In [ ]:
import os

from tqdm import tqdm_notebook as tqdm

from nlppln.utils import get_files

from adhtools.utils import stemmer_xml2df

in_dir = '/home/jvdzwaan/data/tmp/adh/20190325-fiqh-khoja/'

in_files = get_files(in_dir)
in_files = [in_files[0]]

result = {}

for in_file in tqdm(in_files):
    df = stemmer_xml2df(in_file)
    #print(df.columns)
    res = {}
    for name, s in senses.items():
        #print(name, s)
        df['match_{}'.format(name)] = df.apply(lambda row: match_roots(row, s), axis=1)
        res[name] = df['match_{}'.format(name)].sum()
    res['total'] = df.shape[0]
    #print(res)
    file_id = os.path.splitext(os.path.basename(in_file))[0]
    #print(file_id)
    result[file_id] = res
    #break
khod = pd.DataFrame.from_dict(result, orient='index')
khod

In [ ]:
khod.to_csv('senses-fiqh-0179-khoja.csv')

In [ ]:
senses

In [ ]:
import os

from tqdm import tqdm_notebook as tqdm

from nlppln.utils import get_files

from adhtools.utils import stemmer_xml2df

in_dir = '/home/jvdzwaan/data/tmp/adh/20190326-fiqh-isri/'

in_files = get_files(in_dir)
in_files = [in_files[0]]

result = {}

for in_file in tqdm(in_files):
    df = stemmer_xml2df(in_file)
    #print(df.columns)
    res = {}
    for name, s in senses.items():
        #print(name, s)
        df['match_{}'.format(name)] = df.apply(lambda row: match_roots(row, s), axis=1)
        res[name] = df['match_{}'.format(name)].sum()
    res['total'] = df.shape[0]
    #print(res)
    file_id = os.path.splitext(os.path.basename(in_file))[0]
    #print(file_id)
    result[file_id] = res
    #break
isri = pd.DataFrame.from_dict(result, orient='index')
isri

In [ ]:
isri.to_csv('senses-fiqh-0179-isri.csv')

In [ ]:
khod = pd.read_csv('senses-fiqh-new_files-khoja.csv')
khod = khod.rename(columns={'Unnamed: 0': 'BookURI'})
khod = khod.set_index('BookURI')

In [ ]:
khod

In [ ]:
md_file = '/home/jvdzwaan/data/adh-corpora/fiqh_corpus/Meta/Metadata_Fiqh.csv'

md = pd.read_csv(md_file, sep=';|,')
print(md.columns)
md = md.set_index('BookURI')
md

In [ ]:
khoja = pd.concat([khod.copy(), md.copy()], axis=1, sort=True)

In [ ]:
khoja.columns

In [ ]:
def set_schools(row):
    if row['BookSUBJ'] == 'جعفري':
        return 'Shi'
    return 'Sun'


khoja['school'] = khoja.apply(lambda row: set_schools(row), axis=1)

In [ ]:
k_s = khoja.groupby('school').sum()[['hear', 'see', 'touch', 'smell', 'taste', 'total']].T
k_s

In [ ]:
shii_total = k_s.loc['total', 'Shi']
print(shii_total)
k_s['Shi\'i'] = k_s['Shi']/shii_total *100.0

In [ ]:
sunni_total = k_s.loc['total', 'Sun']
print(sunni_total)
k_s['Sunni'] = k_s['Sun']/sunni_total *100.0

In [ ]:
k_s

In [ ]:
alkhalil = pd.concat([alk.copy(), md.copy()], axis=1, sort=True)
alkhalil['school'] = alkhalil.apply(lambda row: set_schools(row), axis=1)
alkhalil

In [ ]:
a_s = alkhalil.groupby('school').sum()[['hear', 'see', 'touch', 'smell', 'taste', 'total']].T
a_s

In [ ]:
shii_total = a_s.loc['total', 'Shi']
print(shii_total)
a_s['Shi\'i'] = a_s['Shi']/shii_total *100.0
sunni_total = a_s.loc['total', 'Sun']
print(sunni_total)
a_s['Sunni'] = a_s['Sun']/sunni_total *100.0
a_s

In [ ]:
k_s

In [ ]:
k_s[['Shi\'i', 'Sunni']].drop(k_s[['Shi\'i', 'Sunni']].tail(1).index).plot(kind='bar', figsize=(7,5), fontsize=12)
plt.legend(fontsize=12)

In [ ]:
a_s[['Shi\'i', 'Sunni']].drop(a_s[['Shi\'i', 'Sunni']].tail(1).index).plot(kind='bar', figsize=(7,5), fontsize=12)
plt.legend(fontsize=12)

In [ ]:
isr = pd.read_csv('senses-fiqh-new_files-isri.csv')
isr = isr.rename(columns={'Unnamed: 0': 'BookURI'})
isr = isr.set_index('BookURI')

isri = pd.concat([isr.copy(), md.copy()], axis=1, sort=True)
isri['school'] = isri.apply(lambda row: set_schools(row), axis=1)
isri

In [ ]:
i_s = isri.groupby('school').sum()[['hear', 'see', 'touch', 'smell', 'taste', 'total']].T
shii_total = i_s.loc['total', 'Shi']
print(shii_total)
i_s['Shi\'i'] = i_s['Shi']/shii_total *100.0
sunni_total = i_s.loc['total', 'Sun']
print(sunni_total)
i_s['Sunni'] = i_s['Sun']/sunni_total *100.0
i_s

In [ ]:
i_s[['Shi\'i', 'Sunni']].drop(i_s[['Shi\'i', 'Sunni']].tail(1).index).plot(kind='bar', figsize=(7,5), fontsize=12)
plt.legend(fontsize=12)

In [ ]:
a_s2 = alkhalil.groupby('BookSUBJ').sum()[['hear', 'see', 'touch', 'smell', 'taste', 'total']].T
a_s2

In [ ]:
for c in a_s2.columns:
    print(c)
    c_total = a_s2.loc['total', c]
    print(c_total)
    a_s2['{} %'.format(c)] = a_s2[c]/c_total *100.0
a_s2

In [ ]:
k_s2 = khoja.groupby('BookSUBJ').sum()[['hear', 'see', 'touch', 'smell', 'taste', 'total']].T
k_s2

In [ ]:
cols = []
for c in k_s2.columns:
    print(c)
    c_total = k_s2.loc['total', c]
    print(c_total)
    n = '{} %'.format(c)
    cols.append(n)
    k_s2[n] = k_s2[c]/c_total *100.0
k_s2

In [ ]:
a_s2[cols].drop(a_s2[cols].tail(1).index).plot(kind='bar')

In [ ]:
k_s2[cols].drop(k_s2[cols].tail(1).index).plot(kind='bar')

In [ ]:
%%time
res = []
titles = []

for title, book in books:
    #print(title)
    b = {}
    for i, roots in query.items():
        num = book[book['root'].isin(roots)].shape[0]
        #print(u'sense {} ({}): {} of {}'.format(i, r, num, total_words))
        b[i] = num
    total_words = book.groupby(['file_name', 'position']).count().shape[0]
    b['other_words'] = total_words-sum(b.values)
    res.append(b)
    titles.append(title)
data = pd.DataFrame(res, index=titles)

In [ ]:
shii = 'شيعي'